<img src='https://radiant-assets.s3-us-west-2.amazonaws.com/PrimaryRadiantMLHubLogo.png' alt='Radiant MLHub Logo' width='300'/>

# 2021 NASA Harvest Rwanda Baseline Model

This notebook walks you through the steps to create a baseline field delineation model for detecting boundaries from Sentinel-2 time-series satellite imagery using a spatio-temporal U-Net model on the [2021 NASA Harvest Rwanda dataset](https://mlhub.earth/data/nasa_rwanda_field_boundary_competition).

In [ ]:
# Importing the needed libraries
import getpass
import glob
import keras
import os
import pickle
import random
from radiant_mlhub import Dataset

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio as rio

import tensorflow as tf
import segmentation_models as sm
from segmentation_models import Unet

from pathlib import Path
from random import choice
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split


from keras import backend as K
from keras.layers import *
from keras.models import *
from keras.models import load_model
from keras.optimizers import *
from keras.preprocessing import image

from tensorflow.keras.layers import *
from tensorflow.keras.losses import *

from sklearn.model_selection import train_test_split

from typing import List, Any, Callable, Tuple

## Authentication
### Create an API Key
Access to the Radiant MLHub API requires an API key. To get your API key, go to [mlhub.earth/profile](mlhub.earth/profile). If you have not used Radiant MLHub before, you will need to sign up and create a new account. Otherwise, sign in. In the **API Keys** tab, you'll be able to create API key(s), which you will need. Do not *share* your API key with others: your usage may be limited and sharing your API key is a security risk.

### Configure the Client
Once you have your API key, you need to configure the `radiant_mlhub` library to use that key. There are a number of ways to configure this (see the [Authentication docs](https://radiant-mlhub.readthedocs.io/en/latest/authentication.html) for details).

For these examples, we will set the `MLHUB_API_KEY` environment variable. Run the cell below to save your API key as an environment variable that the client library will recognize.

*If you are running this notebook locally and have configured a profile as described in the [Authentication docs](https://radiant-mlhub.readthedocs.io/en/latest/authentication.html), then you do not need to execute this cell.*

In [ ]:
os.environ['MLHUB_API_KEY'] = 'YOUR API KEY'

## Data Download
Now that your API credentials have been specified, we will download the data from the [Radiant-MLHub](https://mlhub.earth/) API through Python.

In [ ]:
# Define the dataset id and fetch the dataset".
dataset_id = 'nasa_rwanda_field_boundary_competition'
dataset = Dataset.fetch(dataset_id)

In [ ]:
dataset.download(if_exists='overwrite')

In [ ]:
# Define the image snapshot dimensions and the RBGA Bands
IMG_WIDTH = 256 
IMG_HEIGHT = 256 
IMG_CHANNELS = 4

### The Training Data
We have two sets of data: the `train` and `test` datasets.  

For model development purposes, we will use the training source imagery and label set (`source_train` and `labels_train`) for the model development and use the test set (`source_test`) for model prediction/evaluation.

This saves us memory from training more imagery in our model as well as prevents the model from overfitting. 
Having a separate test dataset which the model hasn't previously seen for prediction also helps us observe the model's results and confirm if they are what we expect.

In [ ]:
train_source_items = f"{dataset_id}/{dataset_id}_source_train"
train_label_items = f"{dataset_id}/{dataset_id}_labels_train"

### Create List of Unique Training IDs
The train source files follow the convention `nasa_rwanda_field_boundary_competition_source_train_ID_YYYY_MM`. We will obtain a list of all the unique IDs for each field where a unique ID is of the format `ID_YYYY_MM`. This makes it easier to load the train dataset.

The corresponding training label files do not inclue any date in the name of the file. Therefore, the training label files follow the convention `nasa_rwanda_field_boundary_competition_label_train_ID`. By striping the `YYYY_MM` component from out from our unique ID (going from `ID_YYYY_MM` to `simply ID`), we can use the unique ID to obtain the corresponding label file for the source imagery.

We will use the function `clean_string` below to get the list of unique IDs.

In [ ]:
def clean_string(s: str) -> str:
    """
    extract the tile id and timestamp from a source image folder
    e.g extract 'ID_YYYY_MM' from 'nasa_rwanda_field_boundary_competition_source_train_ID_YYYY_MM'
    """
    s = s.replace(f"{dataset_id}_source_", '').split('_')[1:]
    return '_'.join(s)

In [ ]:
train_tiles = [clean_string(s) for s in next(os.walk(train_source_items))[1]]

### Normalize the Source Imagery

We will need to normalize our source imagery to remove any pixel values that are greater than 255. We apply the min-max normalisation calculation for each image: $${\text{all pixel values - minimum pixel value} \over \text{maximum pixel value - minimum pixel value}}$$

If we don't, we will be working with distorted images that would add no value to the model during model training.

In [ ]:
def normalize(
    array: np.ndarray
):
    """ normalise image to give a meaningful output """
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)

### Augmentation Techniques

Here, data augmentation will be used to populate the model with data using our normalised images for training to obtain even more accurate results.

We will employ the following data augmentation techniques on the dataset:

- Rotation

- Flipping

- Blurring

These techniques were developed by thanks to the [radix-ai](https://github.com/radix-ai/agoro-field-boundary-detector).

We will input the post-normalization 4-band stack of each image. The 4 bands in this stack will include:

- B01 (Ultra Blue (Coastal and Aerosol))

- B02 (Blue)

- B03 (Green) and

- B04 (Red)

The results of a random source image and its associated label can be seen below.

In [ ]:
tile = random.choice(train_tiles)
print(tile)
bd1 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B01.tif")
bd1_array = bd1.read(1)
bd2 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B02.tif")
bd2_array = bd2.read(1)
bd3 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B03.tif")
bd3_array = bd3.read(1)
bd4 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B04.tif")
bd4_array = bd4.read(1)
b01_norm = normalize(bd1_array)
b02_norm = normalize(bd2_array)
b03_norm = normalize(bd3_array)
b04_norm = normalize(bd4_array)

field = np.dstack((b04_norm, b03_norm, b02_norm, b01_norm))
mask  = rio.open(Path.cwd() / f"{train_label_items}/{dataset_id}_labels_train_{tile.split('_')[0]}/raster_labels.tif").read(1)

We have the following functions that perform the augmentation pieces:

- `t_linear`: performs no augmentation and maintains the same copy of the imagery.

- `t_rotation`: performs rotation on the imagery (e.g by 90 degrees) with multiple rotated copies.

- `t_flip`: performs flipping on the imagery (e.g diagonally, horizontally and vertically) with multiple copies.

- `t_blur`: performs blurring on the imagery using a Gaussian filter with multiple copies.

In [ ]:
def t_linear(
    field: np.ndarray,
    mask: np.ndarray,
    _: int = 0,
) -> Tuple[np.ndarray, np.ndarray]:
    """Apply a linear (i.e. no) transformation and save."""
    return field, mask

def t_rotation(
    field: np.ndarray,
    mask: np.ndarray,
    rot: int,
) -> Tuple[np.ndarray, np.ndarray]:
    """Rotate the data."""
    assert rot in range(0, 3 + 1)
    for _ in range(rot):
        field = np.rot90(field)
        mask = np.rot90(mask)
    return field, mask

def t_flip(
    field: np.ndarray,
    mask: np.ndarray,
    idx: int,
) -> Tuple[np.ndarray, np.ndarray]:
    """Flip the data."""
    assert idx in range(0, 2 + 1)
    if idx == 0: 
        # Diagonal
        field = np.rot90(np.fliplr(field))
        mask = np.rot90(np.fliplr(mask))
    if idx == 1: 
        # Horizontal
        field = np.flip(field, axis=0)
        mask = np.flip(mask, axis=0)
    if idx == 2: 
        # Vertical
        field = np.flip(field, axis=1)
        mask = np.flip(mask, axis=1)
    return field, mask

def t_blur(
    field: np.ndarray,
    mask: np.ndarray,
    sigma: int,
) -> Tuple[np.ndarray, np.ndarray]:
    """Blur the image by applying a Gaussian filter."""
    assert 0 <= sigma <= 10
    sigma_f = 1.0 + (sigma / 10)
    field = np.copy(field)
    for i in range(3):
        field[:, :, i] = gaussian_filter(field[:, :, i], sigma=sigma_f)
    return field, mask

### Helper function for displaying RGB bands of an image

The function `show_image` below allows us to display an RGB image. Below, the `add_subplot` attributes define the left and the right side of the image.

In [ ]:
def show_image(field:np.ndarray, mask:np.ndarray): 
    """
    Show the field and corresponding mask.
    We will just display the RGB bands for simplicity
    """
    fig = plt.figure(figsize=(8,6))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    ax1.imshow(field[:,:,0:3])
    plt.gray()
    ax2.imshow(mask)
    plt.tight_layout()
    plt.show()

### Display Augmented Image Effects
Let's display the image with the augmented effects.

1. The un-augmented image

In [ ]:
show_image(field, mask)

2. The rotated image

In [ ]:
f,m = t_rotation(field, mask, rot=1)
show_image(f,m)

3. The flipped image

In [ ]:
f,m = t_flip(field, mask, idx=0)
show_image(f,m)

4. The blurred image

In [ ]:
f,m = t_blur(field, mask, sigma=5)
show_image(f,m)

We will use the `generate` and `main` functions below to generate and save data augmented images for every source image and its associated label in our training data. We will save the resulting data as pickle `.pkl` files.

We will need to pass into the function: the source image, label and the path to save the augmented images in our directory and a prefix which indicates the id of the imagery. In our case, this is the unique ID we got earlier excluding the timestamp i.e `XX` from `XX_YYYY_MM_DD`.

### Apply Augmentation & Save Images

Now, we need to apply these augmentation techniques to all the source images and their associated labels in our training dataset. 

To generate and save the augmented version of this data, we will use the functions `generate` and `main` below. The resulting data will be saved as `.pkl` (aka pickle) files.

The parameters of these functions are:

- the source image

- the associated label 

- the path to save the augmented images in our directory 

- the image's ID

  - As we discussed above, the ID is the `XX` portion of the `XX_YYYY_MM_DD` portion of the file naming convention.

   

The main function will be called in the sequence below.

In [ ]:
def generate(
    field: np.ndarray,
    mask: np.ndarray,
    write_folder: Path,
    prefix: str = "",
) -> None:
    """
    Generate data augmentations of the provided field and corresponding mask which includes:
     - Linear (no) transformation
     - Rotation
     - Horizontal or vertical flip
     - Gaussian filter (blur)
    :param field: Input array of the field to augment
    :param mask: Input array of the corresponding mask to augment
    :param write_folder: Folder (path) to write the results (augmentations) to
    :param prefix: Field-specific prefix used when writing the augmentation results
    """
    # Generate transformations
    # Dummy data. will be replaced
    f, m = [0,1,2,3], [0,1,2,3]
    # No augmentation
    f[0],m[0] = t_linear(field, mask)
    # Rotation
    f[1],m[1] = t_rotation(field, mask, rot=1)
    # Flipping
    f[2],m[2] = t_flip(field, mask, idx=0)
    # Blurring
    f[3],m[3] = t_blur(field, mask, sigma=5)
    for i in range(len(f)):
        with open(write_folder +'/'+ f"fields/{str(prefix).zfill(2)}_{i}.pkl", 'wb') as handle:
            pickle.dump(f[i], handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(write_folder +'/'+ f"masks/{str(prefix).zfill(2)}_{i}.pkl", 'wb') as handle:
            pickle.dump(m[i], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def main(
    field: List[np.ndarray],
    mask: List[np.ndarray],
    prefix: List[str],
    write_folder: Path,
) -> None:
    """
    Generate and save data augmentations for all the fields and corresponding masks with the following:
     - Linear (no) transformation
     - Rotation
     - Horizontal or vertical flip
     - Gaussian filter (blur)
     - Gamma filter (brightness)
    :param fields: Fields to augment
    :param masks: Corresponding masks to augment
    :param prefixes: Field-specific prefixes corresponding each field
    :param write_folder: Path to write the results (augmentations) to
    """
    generate(
        field=field,
        mask=mask,
        prefix=prefix,
        write_folder=write_folder,
    )

Here, we will use the `main` function which contains the `generate` function for saving the augmented images for all source images and labels.

The following sequence happens for each tile in our train tiles in the block of code below:

1. Get the source images for the four bands (B01, B02, B03, B04) using `rasterio`.

2. Normalise the bands using the normalize function to obtain non-distorted images which make sense for modelling.

3. Obtain the prefix id (`XX`) of the image and corresponding timestamp (`YYYY_MM`) from the tile.

4. Stack the normalised bands on top of each other using `np.dstack`.

5. Use the timestamp as the folder name for our augmented images and the prefix will be used for the filename i.e `/augmented_data/{timestamp}/prefix_{augmentation_id}` (e.g `/augmented_data/2021_01/12_01.pkl`) where `augmentation_id` is the allocated id for the augmented image.

6. Run the augmentation on source images and respective labels.

7. Save results from #6 into the file formats and directories described in #5.

In [ ]:
# Apply augmentation effects to training set
for tile in train_tiles:
    bd1 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B01.tif")
    bd1_array = bd1.read(1)
    bd2 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B02.tif")
    bd2_array = bd2.read(1)
    bd3 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B03.tif")
    bd3_array = bd3.read(1)
    bd4 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B04.tif")
    bd4_array = bd4.read(1)
    b01_norm = normalize(bd1_array)
    b02_norm = normalize(bd2_array)
    b03_norm = normalize(bd3_array)
    b04_norm = normalize(bd4_array)

    # XX_YYYY_MM where XX is the training file id and YYYY_MM is the timestamp
    ids_list  = tile.split('_')
    tile_id   = ids_list[0]
    timestamp = f"{ids_list[1]}_{ids_list[2]}"

    field = np.dstack((b04_norm, b03_norm, b02_norm, b01_norm))
    mask  = rio.open(Path.cwd() / f"{train_label_items}/{dataset_id}_labels_train_{tile_id}/raster_labels.tif").read(1) 

    # Create a folder for the augmented images
    if not os.path.isdir(f"./augmented_data/{timestamp}"):
        os.makedirs(f"./augmented_data/{timestamp}")
    if not os.path.isdir(f"./augmented_data/{timestamp}/fields"):
        os.makedirs(f"./augmented_data/{timestamp}/fields")
    if not os.path.isdir(f"./augmented_data/{timestamp}/masks"):
        os.makedirs(f"./augmented_data/{timestamp}/masks")

    main(
        # Applying augmentation effects
        field  = field,
        mask   = mask,
        prefix = tile_id,
        write_folder = f"./augmented_data/{timestamp}"
    )

In [ ]:
# Get all timestamps
timestamps = next(os.walk(f"./augmented_data"))[1]
# Get all augmented tile ids. can just use one timestamp
augmented_files = next(os.walk(f"./augmented_data/{timestamps[0]}/fields"))[2]
# Time-series image
X = np.empty((len(augmented_files), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS*len(timestamps)), dtype=np.float32)
# Mask for each scene
y = np.empty((len(augmented_files), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.uint8)
i = 0
for file in augmented_files:
    idx = 0
    # Id without .pkl extension
    augmented_id = file.split('.pkl')[0]
    temporal_fields = []
    for timestamp in timestamps:
        with open(f"./augmented_data/{timestamp}/fields/{augmented_id}.pkl", 'rb') as field:
            field = pickle.load(field) 
        X[i][:,:,idx:idx+IMG_CHANNELS] = field
        idx += IMG_CHANNELS
    with open(f"./augmented_data/{timestamp}/masks/{augmented_id}.pkl", 'rb') as mask:
        mask = pickle.load(mask)
    y[i] = mask.reshape(IMG_HEIGHT, IMG_WIDTH, 1)
    i+=1

In [ ]:
# Sanity check
random.randint(0, len(augmented_files))
random_image = random.randint(0, len(augmented_files)-1)
show_image(X[random_image][:,:,0:3], y[random_image])

## Model Training

The U-Net model has been selected in this training for its impressive results over multiple domains in image segmentation.
We will employ a ResNet34 backbone with our spatio-temporal U-Net model.

This model uses our 24 channels (6 timestamps * 4 bands per timestamp) and generates the predicted field boundary per scene.

We will also use an 80:20 train:validation set split for model training.

Since this is a binary segmentation problem (field boundary or no field boundary), we will use the `binary cross_entropy` loss.

We will also use our GPU device for faster model training with better computing power.

Batch normalisation makes training of artificial neural networks faster and more stable through normalization of the layers' inputs by re-centering and re-scaling

In [ ]:
# Activate GPU resource for model training
tf.config.list_physical_devices("GPU")

In [ ]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

In [ ]:
X = preprocess_input(X)

The model's architecture below was tweaked from [sustainlab-group](https://github.com/sustainlab-group/ParcelDelineation/blob/master/models/unet.py) using our number of bands and channels.

In [ ]:
def unet(pretrained_weights = None,input_size = (256,256,4)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return 

In [ ]:
def learning_rate_scheduler(epoch):
    lr = 1e-4
    '''
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 150:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    '''
    print("Set Learning Rate : {}".format(lr))
    return lr

In [ ]:
num_channels = 24
input_shape = (256,256,num_channels)
batch_size = 4

### Model Evaluation Metrics

For the model, we will make use of two key metrics: **Recall** and **F1-score**.

**Recall** evaluates how much of the field boundaries which were labelled were actually predicted correctly while the **F1-score** combines the precision and recall by evaluating the harmonic mean.

Recall is calculated as the number of true positives divided by the total number of true positives and false negatives.

$${\text{True Positives}\over\text{True Positives + False Negatives}}$$

To understand F1-score, first we will define what precision is.

Precision is calculated as the number of true positives divided by the total number of true positives and false positives.

$${\text{True Positives}\over\text{True Positives + False Positives}}$$

Once precision and recall have been calculated for a binary or multiclass classification problem, the two scores can be combined into the calculation of the F1-score.

The traditional F1-score is calculated as follows:

$${\text{2 * Precision * Recall}\over\text{Precision + Recall}}$$

**NOTE** that the recall is the more important metric for this case as we are mostly concerned about the retrieved field boundaries out of the labelled field boundaries.

This [link](https://datascience.stackexchange.com/a/45166) gives a really good implementation on the recall and F1 metrics using Keras. Modifying it a bit into the functions below for our model metrics.

In [ ]:
def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
def f1(y_true, y_pred):
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall_   = recall(y_true, y_pred)
    return 2*((precision*recall_)/(precision+recall_+K.epsilon()))

The U-Net model is initialized with weights from a pre-trained `imagenet` encoder.
The `binary_crossentropy` loss is used with the recall and F1 metrics.
80% of the total data input is allocated to the training set while 20% is allocated to the validation set.

For the sake of compatibility between `tensorflow` and `keras`, eager execution of the `tf.function` decorator function for model training is enabled.

The model is then trained for about `200` epochs to fit well with the data and extract useful information out of it.

It is important to observe the model's performance during training to avoid underfitting and especially, overfitting.
Not using enough epochs doesn't allow the model to learn enough about the dataset, which will lead to underfitting. On the other hand, too many epochs makes the model understand too much about the data, which will lead to overfitting.

In [ ]:
model = None 
model_unet = Unet(BACKBONE, encoder_weights='imagenet')
new_model = keras.models.Sequential()
new_model.add(Conv2D(3, (1,1), padding='same', activation='relu', input_shape=input_shape))
new_model.add(model_unet)
model = new_model 

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=learning_rate_scheduler(0)),
              metrics=[recall,f1])

In [ ]:
model.summary()

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
tf.config.experimental_run_functions_eagerly(True)

In [ ]:
history = model.fit(x=x_train, y=y_train,
              validation_data=(x_val, y_val),
              steps_per_epoch = len(x_train)//batch_size,
              validation_steps = len(x_val)//batch_size,
              batch_size=batch_size, epochs=200)

We will then save the model into the directory we are working in, so that we can load for model prediction on the test dataset.

We named the model `unet_model` and it will be stored in the same relative directory we are working on.

We will then load the model with the defined metrics for prediction on test data set.

In [ ]:
model.save(f"./unet_model.h5")

In [ ]:
model = load_model(f"./unet_model.h5", custom_objects ={"recall":sm.metrics.Recall(threshold=0.5), "f1": f1})

## Model Prediction

The final step is to apply the model prediction on the test dataset.

We will extract the unique ids for the test dataset, just like we did with the train dataset and store them into `test_files`.

In [ ]:
source_test = f"{dataset_id}/{dataset_id}_source_test"
test_tiles = [clean_string(s) for s in next(os.walk(source_test))[1]]

We will then obtain `test_tile_ids` which contains the field overall id excluding the timestamps e.g `10` from `10_2021_12`. Since we already have the list of timestamps in our variable `timestamps`, we only need the field id.

We will then extract the source images (normalising, stacking bands) and store them into `X_test` while `loaded_tiles` tracks the id for each tile that we load in `X_test`.

In [ ]:
test_tile_ids = set()
for tile in test_tiles:
    test_tile_ids.add(tile.split('_')[0])

In [ ]:
X_test = np.empty((len(test_tile_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS*len(timestamps)), dtype=np.float32)
i = 0
loaded_tiles = []
for tile_id in test_tile_ids:
    idx = 0
    for timestamp in timestamps:
        bd1 = rio.open(f"{test_source_items}/{dataset_id}_source_test_{tile_id}_{timestamp}/B01.tif")
        bd1_array = bd1.read(1)
        bd2 = rio.open(f"{test_source_items}/{dataset_id}_source_test_{tile_id}_{timestamp}/B02.tif")
        bd2_array = bd2.read(1)
        bd3 = rio.open(f"{test_source_items}/{dataset_id}_source_test_{tile_id}_{timestamp}/B03.tif")
        bd3_array = bd3.read(1)
        bd4 = rio.open(f"{test_source_items}/{dataset_id}_source_test_{tile_id}_{timestamp}/B04.tif")
        bd4_array = bd4.read(1)
        b01_norm = normalize(bd1_array)
        b02_norm = normalize(bd2_array)
        b03_norm = normalize(bd3_array)
        b04_norm = normalize(bd4_array)
        
        field = np.dstack((b04_norm, b03_norm, b02_norm, b01_norm))
        X_test[i][:,:,idx:idx+IMG_CHANNELS] = field
        idx+=IMG_CHANNELS
    # Track order test tiles are loaded into X to make sure tile id matches 
    loaded_tiles.append(str(tile_id).zfill(2)) 
    i+=1

We will then use the model to predict the field boundaries from the time-series source imagery.
The model's prediction is probabilistic i.e from 0 to 1, where 0 signifies that the model believes no field boundary exist for that particular pixel and 1 signifies the model is confident a field boundary definitely exists for the pixel. That is, a model prediction of 0.3 for a pixel signifies the model believes there is a 30% chance the pixel indicates a field boundary.

Since the label images are either 0 or 1 (a field boundary doesn't exist or it does) for each pixel, We have to choose a threshold where probabilities below that threshold represent 0 (not field boundary) and probabilities above it represent 1 (field boundary). 

In this case, we will choose 0.5 as a fair threshold, which is seen in the below code.

Also, we will reshape the prediction to fit our standard `IMG_HEIGHT * IMG_WIDTH` dimension e.g 256 x 256.

We will then save the prediction into a dictionary where the key is the field id which we stored in `loaded_tiles`.

In [ ]:
predictions_dictionary = {}
for i in range(len(test_tile_ids)):
    model_pred = model.predict(np.expand_dims(X_test[i], 0))
    model_pred = model_pred[0]
    model_pred = (model_pred >= 0.5).astype(np.uint8)
    model_pred = model_pred.reshape(IMG_HEIGHT, IMG_WIDTH)
    predictions_dictionary.update([(str(loaded_tiles[i]), pd.DataFrame(model_pred))])

We will then save the predictions as a `.csv` file as seen below.

In [ ]:
dfs = []
for key, value in predictions_dictionary.items():
    ftd = value.unstack().reset_index().rename(columns={'level_0': 'row', 'level_1': 'column', 0: 'label'})
    ftd['tile_row_column'] = f'Tile{key}_' + ftd['row'].astype(str) + '_' + ftd['column'].astype(str)
    ftd = ftd[['tile_row_column', 'label']]
    dfs.append(ftd)

sub = pd.concat(dfs)
sub

In [ ]:
sub.to_csv(f"./harvest_sample_submission.csv", index = False)